In [1]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from utils import *

In [2]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

### IUCN
#### Removendo espécies sem critério B

In [16]:
# distinct, por causa do land_regions
conn.execute(f"""
    SELECT COUNT(DISTINCT(scientificName_y))
    FROM iucn_complete
""").df()

,count(DISTINCT scientificName_y)
0,122


In [28]:
conn.execute(f"""
    SELECT COUNT(DISTINCT(scientificName_y))
    FROM iucn_complete
    WHERE redListCriteria NOT IN ('D2', 'A4c', 'A4c; B1ab(iii)')
""").df()

,count(DISTINCT scientificName_y)
0,104


In [4]:
conn.execute("DROP TABLE IF EXISTS iucn_only_criteria_b")
conn.execute(f"""
    CREATE TABLE iucn_only_criteria_b AS
    SELECT * FROM iucn_complete
    WHERE redListCriteria NOT IN ('D2', 'A4c', 'A4c; B1ab(iii)')
""")
conn.execute("SELECT COUNT(*) FROM iucn_only_criteria_b").df()

,count_star()
0,491


In [5]:
conn.execute("SELECT COUNT(DISTINCT(scientificName_y)) FROM iucn_only_criteria_b").df()

,count(DISTINCT scientificName_y)
0,104


### Removendo espécie com infos inválidas

In [7]:
conn.execute("""
    SELECT *
    FROM iucn_only_criteria_b
    WHERE scientificName_y='Piper marturetense'
""").df()

,internalTaxonId,scientificName_x,kingdomName,phylumName,className,orderName,familyName,genusName,speciesName,infraType,...,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes,extant,land_regions
0,149308499,Piper marturetense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,marturetense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,"Venezuela, Bolivarian Republic of"


In [8]:
conn.execute("""
    DELETE FROM iucn_only_criteria_b
    WHERE scientificName_y='Piper marturetense'
""").df()

,Count
0,1


### Separando as espécies
#### Separação manual é feita, por causa de divergências

In [3]:
especies_endemicas = [11015485, 11015653, 11015737, 11016854, 11016972, 11017095, 11017382, 11017500, 11017623, 11017756, 11017903, 11018023, 11018277, 11018409, 11018559, 11018675, 11018811, 11018941, 11019059, 11019192, 11019284, 11019424, 11019500, 11019592, 11019748, 11019824, 11019939, 11020152, 11020190, 11020266, 11020344, 11020382, 11020420, 11020572, 11020610, 11020745, 11020821, 146871584, 146871589, 146871594, 146871604, 207997536]
len(especies_endemicas)

42

In [6]:
especies_nativas = [151999900, 42533843, 42533848, 42533853, 120138115, 135681081, 135732370, 136781749, 136786549, 145692398, 146772594, 146779550, 146781476, 146789566, 146871599, 149576848, 149576853, 150006413, 150112286, 150117586, 151947051, 151947133, 151950508, 151963733, 151968027, 151968102, 151970885, 151972126, 151998691, 153317888, 153327552, 153327663, 153333586, 171747790, 176098898, 176522035, 176527104, 179304176, 179304277, 194011025, 194012816, 205617370, 206226925, 206226930, 208355141, 215051506, 218174767, 218175382, 218176505, 218177776, 221260553, 223054223, 226177636, 228551321, 231512300, 239409576, 241878439, 241878531,
 241886450, 242647785, 255924988, 286370469]
len(especies_nativas)

62

In [10]:
conn.execute("DROP TABLE IF EXISTS especies_endemicas")
conn.execute(f"""
    CREATE TABLE especies_endemicas AS
    SELECT * FROM iucn_only_criteria_b
    WHERE assessmentId IN ({",".join([str(x) for x in especies_endemicas])})
""")
conn.execute("SELECT COUNT(DISTINCT(scientificName_y)) FROM especies_endemicas").df()

,count(DISTINCT scientificName_y)
0,42


In [19]:
# 61, porque excluiu a Piper marturetense
conn.execute("DROP TABLE IF EXISTS especies_nativas")
conn.execute(f"""
    CREATE TABLE especies_nativas AS
    SELECT * FROM iucn_only_criteria_b
    WHERE assessmentId IN ({",".join([str(x) for x in especies_nativas])})
""")
conn.execute("SELECT COUNT(DISTINCT(scientificName_y)) FROM especies_nativas").df()

,count(DISTINCT scientificName_y)
0,61


###  Removendo espécies que são da América Central

In [37]:
sql = f"""
    SELECT DISTINCT(*)
    FROM powo
    WHERE native_countries NOT LIKE '%Brazil%' AND
    native_countries NOT LIKE '%French Guiana%' AND
    native_countries NOT LIKE '%Guyana%' AND
    native_countries NOT LIKE '%Suriname%' AND
    native_countries NOT LIKE '%Venezuela%' AND
    native_countries NOT LIKE '%Bolivia%' AND
    native_countries NOT LIKE '%Colombia%' AND
    native_countries NOT LIKE '%Galápagos%' AND
    native_countries NOT LIKE '%Ecuador%' AND
    native_countries NOT LIKE '%Peru%' AND
    native_countries NOT LIKE '%Argentina%' AND
    native_countries NOT LIKE '%Chile%' AND
    native_countries NOT LIKE '%Paraguai%' AND
    native_countries NOT LIKE '%Uruguai%';
"""
res = conn.execute(sql).df()
res.head()

,species,name,url,publicacao,native_countries,native_countries_count,iucn,iucn_code,iucn_full_name,status_species,native_range
0,Piper hispidum,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,First published in Acta Phytotax. Geobot. 4: 2...,Laos; Taiwan,2,None,None,None,This species is accepted,"The native range of this species is Laos, Taiw..."
1,Piper friedrichsthalii,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,"First published in A.P.de Candolle, Prodr. 16(...",Costa Rica; Nicaragua; Panamá,3,LC - least,LC,least,This species is accepted,The native range of this species is Nicaragua ...
2,Piper grande,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,"First published in Trans. Linn. Soc. London, B...",New Guinea,1,None,None,None,None,None
3,Piper fimbriulatum,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,First published in Bull. Soc. Roy. Bot. Belgiq...,Costa Rica,1,NT - near,NT,near,This species is accepted,The native range of this species is Costa Rica...
4,Piper reticulatum,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,"First published in Nomencl. Bot., ed. 2, 2: 34...",Mexico Central; Mexico Gulf; Mexico Northeast;...,6,None,None,None,This species is accepted,The native range of this species is Mexico. It...


In [42]:
conn.execute(f"""
    SELECT DISTINCT(scientificName_y)
    FROM especies_nativas
    WHERE scientificName_y IN ({",".join(["\'%s\'" % x for x in res["species"].tolist()])});
""").df()

,scientificName_y
0,Piper friedrichsthalii
1,Piper corrugatum
2,Piper curtispicum


In [43]:
conn.execute(f"""
    DELETE
    FROM especies_nativas
     WHERE scientificName_y IN ({",".join(["\'%s\'" % x for x in res])});
""").df()

,Count
0,13


### Atualiza o nome de uma espécie pelo seu sinônimo

In [44]:
conn.execute("""
    SELECT *
    FROM especies_nativas
    WHERE scientificName_y='Piper bogotense';
""").df()

,internalTaxonId,scientificName_x,kingdomName,phylumName,className,orderName,familyName,genusName,speciesName,infraType,...,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes,extant,land_regions
0,42532896,Piper bogotense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,bogotense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Peru
1,42532896,Piper bogotense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,bogotense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador (mainland)
2,42532896,Piper bogotense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,bogotense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Colombia
3,42532896,Piper bogotense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,bogotense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,"Venezuela, Bolivarian Republic of"
4,42532896,Piper bogotense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,bogotense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Panama
5,42532896,Piper bogotense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,bogotense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Colombia (mainland)
6,42532896,Piper bogotense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,bogotense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador
7,42532896,Piper bogotense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,bogotense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Venezuela (mainland)


In [45]:
conn.execute("""
    UPDATE especies_nativas
    SET scientificName_y='Piper barbatum'
    WHERE scientificName_y='Piper bogotense';
""").df()

,Count
0,8


### Espécies nativas na lista de endêmicas

In [6]:
especies_erradas = ["Piper brachystylum", "Piper campii", "Piper densiciliatum", "Piper disparipilum", "Piper longicaudatum", "Piper nanegalense", "Piper regale", "Piper tucumanum"]
len(especies_erradas)

8

In [10]:
conn.execute(f"""
    SELECT *
    FROM especies_endemicas
    WHERE scientificName_y IN ({",".join(["\'%s\'" % x for x in especies_erradas])});
""").df()

,internalTaxonId,scientificName_x,kingdomName,phylumName,className,orderName,familyName,genusName,speciesName,infraType,...,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes,extant,land_regions
0,45821,Piper brachystylum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,brachystylum,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador
1,45823,Piper campii,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,campii,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador
2,45829,Piper densiciliatum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,densiciliatum,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador
3,45831,Piper disparipilum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,disparipilum,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador
4,45844,Piper longicaudatum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,longicaudatum,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador
5,45849,Piper nanegalense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,nanegalense,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador
6,45858,Piper regale,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,regale,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Ecuador
7,207997286,Piper tucumanum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,tucumanum,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,False,"Bolivia, Plurinational States of"
8,207997286,Piper tucumanum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,tucumanum,NaN,...,None,Terrestrial,None,Neotropical,NaN,False,False,Global,True,Argentina


In [15]:
conn.execute(f"""
    INSERT INTO especies_nativas (internalTaxonId, scientificName_x, kingdomName, phylumName, className, orderName, familyName, genusName, speciesName, infraType, infraName, infraAuthority, subpopulationName, authority, taxonomicNotes, assessmentId, scientificName_y, redlistCategory, redlistCriteria, yearPublished, assessmentDate, criteriaVersion, language, rationale, habitat, threats, population, populationTrend, range, useTrade, systems, conservationActions, realm, yearLastSeen, possiblyExtinct, possiblyExtinctInTheWild, scopes, extant, land_regions)
    SELECT internalTaxonId, scientificName_x, kingdomName, phylumName, className, orderName, familyName, genusName, speciesName, infraType, infraName, infraAuthority, subpopulationName, authority, taxonomicNotes, assessmentId, scientificName_y, redlistCategory, redlistCriteria, yearPublished, assessmentDate, criteriaVersion, language, rationale, habitat, threats, population, populationTrend, range, useTrade, systems, conservationActions, realm, yearLastSeen, possiblyExtinct, possiblyExtinctInTheWild, scopes, extant, land_regions
    FROM especies_endemicas
    WHERE scientificName_y IN ({",".join(["\'%s\'" % x for x in especies_erradas])});
""").df()

,Count
0,9


### Piper nanegalense é sinônimo Piper obliquum

In [3]:
conn.execute("""
    UPDATE especies_nativas
    SET scientificName_y='Piper obliquum'
    WHERE scientificName_y='Piper nanegalense';
""").df()

,Count
0,1


In [17]:
conn.execute(f""" SELECT COUNT(DISTINCT(scientificName_y)) FROM especies_nativas """).df()

,count(DISTINCT scientificName_y)
0,66


In [18]:
conn.execute(f""" SELECT COUNT(DISTINCT(scientificName_y)) FROM especies_endemicas """).df()

,count(DISTINCT scientificName_y)
0,42


In [4]:
conn.close()